# 必須

In [323]:
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import sys 
sys.path.append("..")

from utils import *

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

In [272]:

df = read_new_csv()
df_train, df_test = df_split(df)
#date_df = read_csv(date)

../feature/10301905.csv was read


In [210]:
#df_all = read_csv(ALL)
#train_df_,test_df_ = df_split(df_all)
df

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,grade.1,credit_score,application_type,...,credit_1,credit_2,credit_3,credit_4,loan_grade,loan_0,loan_1,loan_2,loan_3,loan_4
0,0,1742.228656,5,7.992879,4,8,debt_consolidation,A4,677.196371,Individual,...,1,0,0,0,loan_2,0,0,1,0,0
1,1,630.300979,3,13.604302,18,6,debt_consolidation,D3,655.143270,Individual,...,0,0,0,0,loan_0,1,0,0,0,0
2,2,1606.895116,5,24.674460,15,10,credit_card,C5,680.532978,Individual,...,1,0,0,0,loan_2,0,0,1,0,0
3,3,1120.716185,3,13.407068,11,1,credit_card,C1,659.544561,Individual,...,0,0,0,0,loan_1,0,1,0,0,0
4,4,688.615551,3,18.118501,12,10,other,C2,654.822203,Individual,...,0,0,0,0,loan_0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,17495,2495.568495,3,11.806962,8,10,credit_card,B3,679.886560,Individual,...,1,0,0,0,loan_3,0,0,0,1,0
24996,17496,636.151639,3,11.449207,7,0,car,B2,660.318169,Individual,...,0,0,0,0,loan_0,1,0,0,0,0
24997,17497,2398.851088,5,24.501209,25,8,debt_consolidation,E5,657.758005,Individual,...,0,0,0,0,loan_3,0,0,0,1,0
24998,17498,1010.424260,3,8.196667,5,0,medical,A5,707.729934,Individual,...,0,0,1,0,loan_1,0,1,0,0,0


## データの分割

In [211]:
df.tail()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,grade.1,credit_score,application_type,...,credit_1,credit_2,credit_3,credit_4,loan_grade,loan_0,loan_1,loan_2,loan_3,loan_4
24995,17495,2495.568495,3,11.806962,8,10,credit_card,B3,679.886560,Individual,...,1,0,0,0,loan_3,0,0,0,1,0
24996,17496,636.151639,3,11.449207,7,0,car,B2,660.318169,Individual,...,0,0,0,0,loan_0,1,0,0,0,0
24997,17497,2398.851088,5,24.501209,25,8,debt_consolidation,E5,657.758005,Individual,...,0,0,0,0,loan_3,0,0,0,1,0
24998,17498,1010.424260,3,8.196667,5,0,medical,A5,707.729934,Individual,...,0,0,1,0,loan_1,0,1,0,0,0
24999,17499,1106.931375,3,7.097667,1,1,credit_card,A1,716.322383,Individual,...,0,0,1,0,loan_1,0,1,0,0,0


In [273]:
df.columns

Index(['id', 'loan_amnt', 'term', 'interest_rate', 'grade',
       'employment_length', 'purpose', 'grade.1', 'credit_score',
       'application_type', 'loan_status', 'is_train', 'amnt_per_year',
       'grade_ordinal', 'interest', 'sum_amnt', 'grade_bin', 'amntpyear_bin',
       'interest_grade', 'interest_0', 'interest_1', 'interest_2',
       'interest_3', 'interest_4', '0', '1', '2', '3', '4', 'purpose_major',
       'credit_grade', 'credit_0', 'credit_1', 'credit_2', 'credit_3',
       'credit_4', 'loan_grade', 'integral_score'],
      dtype='object')

In [274]:
from  sklearn.model_selection  import train_test_split
#using = ["age", "job", "marital", "education", "default", "balance_log_cut", "housing", "loan","contact",  "duration_log", "campaign_bin", "pdays", "previous_bin", "poutcome", "month_day", "is_job_kesson","is_education_kesson","is_poutcome_kesson","is_contact_kesson" ]
using = ['loan_amnt', 'term', 'interest_4','interest_0', 'grade',
       'credit_1', 'purpose_major','credit_0','credit_3','credit_4',]
y = df_train[TARGET].values
X = df_train[using].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
y

array([0., 0., 1., ..., 0., 0., 0.])

# モデル

In [164]:
SEED = 10
# 必要なもののインポート
## 予測モデル
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

# 実験

## 検証　with F1 0.25492668483979

In [225]:
# 使用するモデル
clf = RandomForestClassifier(n_estimators=100, random_state=SEED)

# Stratified k-Fold で汎化性能を評価する
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# 評価する指標
score_funcs = ['f1']
# Cross Validation で検証する
scores = cross_validate(clf, X, y, cv=skf, scoring=score_funcs)
# 得られた指標を出力する
print('f1:', scores['test_f1'].mean(),"\n")
print(scores)

f1: 0.29933121509723737 

{'fit_time': array([1.13690686, 1.16383195, 1.12103391, 1.13153815, 1.12768507]), 'score_time': array([0.07257295, 0.07016015, 0.07185411, 0.07010603, 0.07291007]), 'test_f1': array([0.3166784 , 0.30133146, 0.28531272, 0.28671329, 0.30662021])}


0.29933121509723737:0.4166165

## base line with RFC　（0.９３２）

In [186]:
rf = RandomForestClassifier()
rf.fit(X,y)

RandomForestClassifier()

In [187]:
prd = rf.predict_proba(df_test[using])
prd_df = pd.DataFrame(prd)[1]
prd_df = np.where(prd_df < 0.34, 0, 1)
prd_df = pd.DataFrame(prd_df)

In [188]:
np.where(prd_df < 0.34, 0, 1)

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [189]:
save_feature(prd_df)

../feature/10301343.csv
save completed


In [208]:
from pprint import pprint
fti = rf.feature_importances_ 
feature_list = list(zip(using, fti))
feature_list.sort(key=lambda x: x[1], reverse = True)
max_len_feature = max(map( lambda s: len(s), using))
print('Feature Importances:')
#pprint(feature_list)
for tp in feature_list:
    print("{:{}}:{:.3}".format(tp[0],max_len_feature+1, tp[1]))
print("top10")
for s in feature_list[0:11]:
    print(f"'{s[0]}',",end = '')

Feature Importances:
credit_score  :0.38
interest_1    :0.37
interest_0    :0.0974
interest_2    :0.0505
term          :0.0162
purpose_major :0.00919
loan_amnt     :0.00779
interest_4    :0.00722
top10
'credit_score','interest_1','interest_0','interest_2','term','purpose_major','loan_amnt','interest_4',

## base line with LGBM（0.938）

In [342]:
params = {
    # 二値分類問題
    'objective': 'binary',
    # AUC の最大化を目指す
    'metric': 'binary',
    # Fatal の場合出力
    'verbosity': -1,
    
}
lgb_train = lgb.Dataset(X, y)

In [232]:
model = lgb.train(params, lgb_train,
                  verbose_eval=50,  # 50イテレーション毎に学習結果出力
                  num_boost_round=100,  # 最大イテレーション回数指定
                 )

In [233]:
y_pred_lgb = model.predict(df_test[using])

In [53]:
prd_df = pd.DataFrame(y_pred_lgb)
#df_prd = prd_df.iloc[:, [1]]
prd_df.to_csv(SUBMIT+"/hoo3.csv")
#to_submit_csv(prd_df)

In [51]:
df_prd = prd_df.iloc[:, [0]]
boo = np.squeeze(np.array(df_prd))

hoo = np.add(boo, y_pred_lgb)/2

In [52]:
to_submit_csv(pd.DataFrame(hoo))

KeyError: 'ID'

In [234]:
np.mean(y_pred_lgb)

0.2125967076472296

In [239]:
print(np.mean(np.where(y_pred_lgb < 0.3, 0, 1)))

0.22813333333333333


In [240]:
#np.where(y_pred_lgb < 0.35, 0, 1)
prd_df = pd.DataFrame(np.where(y_pred_lgb < 0.3, 0, 1))
save_feature(prd_df)

../feature/10301551.csv
save completed


## かさまし

In [315]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

In [316]:
len(y_resampled)

19270

In [335]:
# 使用するモデル
lgb_train_resapled = lgb.Dataset(X_resampled, y_resampled)
gbm = lgb.train(params, lgb_train_resapled,
                  verbose_eval=50,  # 50イテレーション毎に学習結果出力
                  num_boost_round=50,# 最大イテレーション回数指定
                feval=lgb_f1_score,
                 )

y_repred_proba = gbm.predict(X_test)
y_repred=np.where(y_repred_proba < 0.35, 0, 1)
print(np.mean(y_repred))
print(f1_score(y_repred, y_test))

0.29028571428571426
0.4145427286356821


In [341]:
# 使用するモデル
evals_result = {}
lgb_train_check = lgb.Dataset(X_train, y_train)
gbm = lgb.train(params, lgb_train_check,
                  verbose_eval=10,  # 50イテレーション毎に学習結果出力
                  num_boost_round=100,  # 最大イテレーション回数指定
                feval=lgb_f1_score,
                evals_result=evals_result,
                 )
y_pred_proba = gbm.predict(X_test)
print(evals_result)
y_pred=np.where(y_pred_proba < 0.33, 0, 1)
print(np.mean(y_pred))
print(f1_score(y_pred, y_test))
print()
lgb.plot_metric(evals_result, metric='f1')

{}
0.17485714285714285
0.3724539282250242



ValueError: eval results cannot be empty.

0.3157443157443157:0.4166165

＝＞　か覚ましした結果よくなっている。

In [116]:
np.mean(y_repred)

0.16038095238095237

## ハイパラチューニング

In [343]:
lgb_train = lgbo.Dataset(X_train, y_train)
lgb_eval = lgbo.Dataset(X_test, y_test, reference=lgb_train)

# 使用するモデル
gbm = lgbo.train(params, lgb_train, valid_sets=lgb_eval,
                  verbose_eval=50,  # 50イテレーション毎に学習結果出力
                  num_boost_round=50,  # 最大イテレーション回数指定
                 )

y_repred_proba = gbm.predict(X_test)
y_repred=np.where(y_repred_proba < 0.35, 0, 1)
print(np.mean(y_repred))
print(f1_score(y_repred, y_test))

[I 2021-10-30 19:58:22,820] A new study created in memory with name: no-name-8231cd27-bde3-41ff-95a7-90da8e03057b
feature_fraction, val_score: 0.476752:  29%|##8       | 2/7 [00:00<00:00,  7.81it/s]

[50]	valid_0's binary_logloss: 0.476752
[50]	valid_0's binary_logloss: 0.477523


feature_fraction, val_score: 0.476752:  71%|#######1  | 5/7 [00:00<00:00, 10.11it/s]

[50]	valid_0's binary_logloss: 0.476952
[50]	valid_0's binary_logloss: 0.477493
[50]	valid_0's binary_logloss: 0.477996


num_leaves, val_score: 0.476752:   0%|          | 0/20 [00:00<?, ?it/s]

...


[50]	valid_0's binary_logloss: 0.476149
[50]	valid_0's binary_logloss: 0.47643


min_data_in_leaf, val_score: 0.475571:  60%|######    | 3/5 [00:00<00:00, 12.76it/s]

[50]	valid_0's binary_logloss: 0.475936
[50]	valid_0's binary_logloss: 0.476009
[50]	valid_0's binary_logloss: 0.476064


min_data_in_leaf, val_score: 0.475571: 100%|##########| 5/5 [00:00<00:00, 11.75it/s]


[50]	valid_0's binary_logloss: 0.4761
[50]	valid_0's binary_logloss: 0.476464
0.14685714285714285
0.3676240208877285


# 提出ファイルの作成

In [311]:
from imblearn.over_sampling import SMOTE
import optuna.integration.lightgbm as lgbo
sm_sub = SMOTE()
X_sub, y_sub = sm.fit_resample(X, y)

In [312]:
# 使用するモデル
lgb_train_sub = lgb.Dataset(X_sub, y_sub)
gbm_sub = lgb.train(params, lgb_train_sub,
                  verbose_eval=50,  # 50イテレーション毎に学習結果出力
                  num_boost_round=50,  # 最大イテレーション回数指定
                 )


y_pred_sub = gbm_sub.predict(df_test[using])

In [313]:
prd_df = pd.DataFrame(np.where(y_pred_sub < 0.35, 0, 1))
print(np.mean(prd_df[0]))

0.3248


In [314]:
save_feature(prd_df)

../feature/10301941.csv
save completed
